In [1]:
import random
import numpy as np
import argparse
from itertools import groupby

In [2]:
NUM_PLIES =16
MUTATION_RATE=.1
POP_SIZE = 100
NUM_ITERATIONS=1000

In [3]:
# Fitness function
def fitness(seq):
    # 1. Use balanced and symmetric layups
    num_45deg_upper = sum(ply[1] == 1 for ply in seq[:NUM_PLIES//2])
    num_45deg_lower = sum(ply[1] == 2 for ply in seq[NUM_PLIES//2:])
    if num_45deg_upper != num_45deg_lower:
        return float('-inf')
    
    # 2. Intersperse ply orientations
    num_0deg = sum(ply[1] == 0 for ply in seq)
    num_45deg = sum(ply[1] == 1 or ply[1] == 2 for ply in seq)
    num_90deg = sum(ply[1] == 3 for ply in seq)
    if num_0deg < NUM_PLIES//10 or num_45deg < NUM_PLIES//10 or num_90deg < NUM_PLIES//10:
        return float('-inf')

    # 3. Minimize groupings of plies with the same orientation
    group_lengths = [len(list(group)) for key, group in groupby(seq, key=lambda ply: ply[1])]
    if max(group_lengths) > 1:
        return float('-inf')

    # 4. Alternate +θ° and -θ° plies except for the closest ply on either side of the mid-plane
    if seq[NUM_PLIES//2-1][1] == seq[NUM_PLIES//2][1] or seq[NUM_PLIES//2][1] == seq[NUM_PLIES//2+1][1]:
        return float('-inf')
    for i in range(NUM_PLIES-3):
        if seq[i][1] == seq[i+1][1] and seq[i+1][1] == seq[i+2][1]:
            return float('-inf')

    # 5. Separate groups of same orientation by 45° plies (e.g, tape plies of 90° apart by at least one ply of 45° apart)
    for i in range(NUM_PLIES-2):
        if seq[i][1] == seq[i+1][1] == seq[i+2][1]:
            if seq[i][1] == 3:
                if i == NUM_PLIES//2-2:
                    if seq[i+3][1] == 1 or seq[i+3][1] == 2:
                        return float('-inf')
                elif i == NUM_PLIES//2-1:
                    if seq[i-1][1] == 1 or seq[i-1][1] == 2 or seq[i+2][1] == 1 or seq[i+2][1] == 2:
                        return float('-inf')
                elif i == NUM_PLIES//2:
                    if seq[i-2][1] == 1 or seq[i-2][1] == 2 or seq[i+1][1] == 1 or seq[i+1][1] == 2:
                        return float('-inf')
                    
    # 6. 10% design rule
    orientations = [0, 0, 0, 0] # count of 0°, 45°, -45°, 90°
    for i in range(NUM_PLIES):
        orientations[seq[i][1]] += 1
        if min(orientations) < NUM_PLIES * 0.1:
            return float('-inf')
               
    # 7. At a free edge, do not locate tape plies with fibers orientated perpendicular to the edge
    if seq[0][1] == 3 or seq[NUM_PLIES-1][1] == 3:
        if seq[NUM_PLIES//2][1] == 1 or seq[NUM_PLIES//2][1] == 2:
            return float('-inf')
        
    # 8. Locate tape plies with fibers oriented in the primary load direction such that there is a least three plies between them and the laminate surface.
    for i in range(NUM_PLIES):
        if seq[i][1] == 0:
            if i < 3 or i > NUM_PLIES - 4:
                return float('-inf')
            if seq[i-1][1] != 45 or seq[i-2][1] != -45 or seq[i-3][1] != 45:
                return float('-inf')
            if seq[i+1][1] != 45 or seq[i+2][1] != -45 or seq[i+3][1] != 45:
                return float('-inf')
    return sum(ply[0] for ply in seq)  # return the sum

In [4]:
# Mutation function
def mutate(seq):
    for i in range(NUM_PLIES):
        if random.random() < MUTATION_RATE:
            seq[i] = (random.uniform(0, 1), seq[i][1])
    return seq

In [5]:
def select_parents(population, fitnesses):
    total_fitness = sum(fitnesses)
    probs = [fit/total_fitness for fit in fitnesses]
    parent1_idx = random.choices(range(POP_SIZE), weights=probs)[0]
    parent2_idx = random.choices(range(POP_SIZE), weights=probs)[0]
    return population[parent1_idx], population[parent2_idx]

In [6]:
# Crossover function
def crossover(parent1, parent2):
    # Choose a random midpoint
    midpoint = random.randint(1, NUM_PLIES-1)
    # Swap the sub-sequences
    child1 = parent1[:midpoint] + parent2[midpoint:]
    child2 = parent2[:midpoint] + parent1[midpoint:]
    return child1, child2

In [7]:
# Generate the initial stacking sequence population
def generate_initial_population():
    population = []
    for i in range(POP_SIZE):
        seq = []
        for j in range(NUM_PLIES):
            ply = (random.uniform(0, 1), j % 4)
            seq.append(ply)
        population.append(seq)
    return population

In [9]:
# Find the best stacking sequence
def genetic_algorithm():
    population = generate_initial_population()
    for i in range(NUM_ITERATIONS):
        fitnesses = [fitness(seq) for seq in population]
        best_fitness = max(fitnesses)
        best_seq = population[fitnesses.index(best_fitness)]
        print(f'Generation {i+1}: Best Fitness = {best_fitness:.2f}')

        next_population = [best_seq]
        for j in range(POP_SIZE-1):
            parent1, parent2 = select_parents(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_population.append(child)
        population = next_population

    return best_seq

In [10]:
best_seq = genetic_algorithm()
print('Best Sequence:', best_seq)

Generation 1: Best Fitness = -inf


ValueError: Total of weights must be finite